## OpenAI Function Calling

---

## Setup

In [122]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

deployment_id = "gpt40125"

## Define the Function

In [123]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "32",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [124]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

## Function Calling

### Prompt related to the function

In [125]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [126]:
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions
)
response

<OpenAIObject chat.completion id=chatcmpl-9CmDqmquDZKnuJnzFYi2LUB4Aurgz at 0x1fb4e8f7f50> JSON: {
  "choices": [
    {
      "content_filter_results": {},
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\"location\":\"Boston, MA\",\"unit\":\"fahrenheit\"}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1712832330,
  "id": "chatcmpl-9CmDqmquDZKnuJnzFYi2LUB4Aurgz",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
    

In [127]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x1fb4e930110> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\"location\":\"Boston, MA\",\"unit\":\"fahrenheit\"}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

The function call does not actually call the function, but returns the necessary info to call the function. We still need to manually call the function.

In [128]:
args = json.loads(response_message["function_call"]["arguments"])
get_current_weather(args)

'{"location": {"location": "Boston, MA", "unit": "fahrenheit"}, "temperature": "32", "unit": "celsius", "forecast": ["sunny", "windy"]}'

### Prompt not related to the function

In [129]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [130]:
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    temperature=0,
    messages=messages,
    functions=functions
)
response

<OpenAIObject chat.completion id=chatcmpl-9CmDtbYYricA9xAhUKfk7ZML9sud2 at 0x1fb4e8f75f0> JSON: {
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1712832333,
  "id": "chatcmpl-9CmDtbYYricA9xAhUKfk7ZML9sud2",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severi

In [131]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x1fb4e932b10> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}

This time model did not use the function since the prompt was not related to the function.

## Parameter to control using function

Parameter `function_call` can take the following values:
- `auto`: The model will decide whether to use the function or not. This is also the default value.
- `none`: The model will not use the function.
- `{"name": "<function_name>"}`: The model will always use the function with the given name.

### `auto`

In [132]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call="auto"
)
response["choices"][0]["message"]

<OpenAIObject at 0x1fb4e933b30> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}

### `none`

In [133]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    functions=functions,
    function_call="none"
)
response["choices"][0]["message"]

<OpenAIObject at 0x1fb4e9334d0> JSON: {
  "content": "To provide you with accurate weather information, could you specify if you'd like the temperature in Celsius or Fahrenheit?",
  "role": "assistant"
}

### `{"name": "<function_name>"}`

In [134]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
response["choices"][0]["message"]

<OpenAIObject at 0x1fb4e933b90> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\"location\":\"Boston, MA\",\"unit\":\"fahrenheit\"}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

What happens when we force the model to use the function and the prompt is not related to the function?

In [135]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
response["choices"][0]["message"]

<OpenAIObject at 0x1fb4e932f30> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\"location\":\"San Francisco, CA\",\"unit\":\"celsius\"}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

Model got confused and made up things, there was nothing related to the function in the prompt. Since, we forced the model to use the function, it made up things.

## Token Usage

Function calling increases the token count becasue of additional content.

### Utlity function to extract relevant details

In [136]:
from typing import Dict, List, Any


DETAILS_DICT: Dict[str, List[Any]] = {
    "mode": [],
    "prompt_mssg": [],
    "response_mssg": [],
    "prompt_tokens": [],
    "completion_tokens": [],
    "total_tokens": [],
}

def save_details(mode: str, prompt_mssg: Any, response: Any) -> None:
    global DETAILS_DICT
    DETAILS_DICT["mode"].append(mode)
    DETAILS_DICT["prompt_mssg"].append(prompt_mssg)
    DETAILS_DICT["response_mssg"].append(response["choices"][0]["message"])
    usage = response["usage"]
    DETAILS_DICT["prompt_tokens"].append(usage["prompt_tokens"])
    DETAILS_DICT["completion_tokens"].append(usage["completion_tokens"])
    DETAILS_DICT["total_tokens"].append(usage["total_tokens"])


### Call with different modes

In [137]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [138]:
# no function call

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
)

save_details("no_function_call", messages, response)

In [139]:
# function call - none

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call="none"
)

save_details("none", messages, response)

In [140]:
# function call - auto

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call="auto"
)

save_details("auto", messages, response)

In [141]:
# function call - forced

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions,
    function_call={"name": "get_current_weather"}
)

save_details("forced", messages, response)

### Display details

In [142]:
import pandas as pd

df = pd.DataFrame(DETAILS_DICT)
pd.set_option("display.max_colwidth", None)
df

,mode,prompt_mssg,response_mssg,prompt_tokens,completion_tokens,total_tokens
0,no_function_call,"[{'role': 'user', 'content': 'hi!'}]","{'content': 'Hello! How can I assist you today?', 'role': 'assistant'}",9,9,18
1,none,"[{'role': 'user', 'content': 'hi!'}]","{'content': 'Hello! How can I assist you today?', 'role': 'assistant'}",77,9,86
2,auto,"[{'role': 'user', 'content': 'hi!'}]","{'content': 'Hello! How can I assist you today?', 'role': 'assistant'}",76,10,86
3,forced,"[{'role': 'user', 'content': 'hi!'}]","{'content': None, 'function_call': {'arguments': '{""location"":""New York, NY"",""unit"":""fahrenheit""}', 'name': 'get_current_weather'}, 'role': 'assistant'}",86,13,99


## Pass function call results to model

### Do the function call

In [145]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
    functions=functions
)
response_mssg = response["choices"][0]["message"]
response_mssg

<OpenAIObject at 0x1fb4f4a04d0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\"location\":\"Boston, MA\",\"unit\":\"fahrenheit\"}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

### Append the assistant's message to the prompt

In [146]:
messages.append(response_mssg)

### Append the function call results to the prompt

In [147]:
args = json.loads(response_mssg['function_call']['arguments'])
observation = get_current_weather(args)

In [148]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation,
    }
)

### Call the model

In [150]:
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=messages,
    temperature=0,
)
response_mssg = response["choices"][0]["message"]
response_mssg

<OpenAIObject at 0x1fb4f4a05f0> JSON: {
  "content": "The current weather in Boston, MA is sunny and windy with a temperature of 32\u00b0C.",
  "role": "assistant"
}